Load packages

In [1]:
import numpy as np
import radvel
import pickle
import pandas as pd
import sys
import re
import dynesty
DATADIR = "../data/"
sys.path+= ["../"]
import matplotlib.pyplot as plt
%matplotlib inline

unclosed file <_io.TextIOWrapper name='/Users/shadden/anaconda/lib/python3.6/site-packages/astropy/extern/bundled/six.py' mode='r' encoding='utf-8'>


In [2]:
from ResonantRV_Utils import get_acr_like
from ResonantPairModel import ACRModelPrior, ACRModelPriorTransform

# Example RV Fitting

This notebook constains examples demonstrating our ACR model and RV fitting methods. We conduct MCMC and nested sampling fits to compute posterior samples and estimate the Baysian evidence of the model. We generate `radvel.likelihood.CompositeLikelihood` and `radvel.posterior.Posterior` objects for the model under consideration.  The posterior objects are used in a MCMC fit to generate posterior samples.  The likelihood objects are used along side a transformation of the unit cube to run a nested sampling algorithm using `dynesty`.


We begin by selecting our example system as  **HD 33844**

In [7]:
ObservationsDF = pd.read_pickle(DATADIR+"All_Observations.pkl")
system='HD 33844'
Observations = ObservationsDF.query('system==@system')

In [11]:
HostStarInfoFile = "../data/HostStarInfo.pkl"
with open(HostStarInfoFile,"rb") as fi:
    HostStarData=pickle.load(fi)
HostStarData[system][0]


1.84

In [14]:
HostStarData['HD 45364']

[0.82, 'Correia2009']

# ACR model

Generate likelihood object for ACR model

In [9]:
res_j,res_k = 5,2
acr_model_like = get_acr_like(Observations,res_j,res_k,ACR_data_file="../ACR_locations_data.pkl")

Set initial values of 'like' object parameters to the values stored in the full model (where applicable). Parameters not shared with the full model assume their default values. We'll use this as an initial guess for max-likelihood fitting. 

In [ ]:
for key,par in acr_model_like.params.items():
    if key in full_model_like.params.keys():
        par.value = full_model_like.params[key].value

Create posterior object for radvel MCMC fit

In [ ]:
acr_model_post = radvel.posterior.Posterior(acr_model_like)
acrpriors = [
    ACRModelPrior(),
    radvel.prior.Jeffreys('k1',0.2 * k1best, 5 * k1best ),
    radvel.prior.Jeffreys('m2_by_m1',0.1,10)
]
acr_model_post.priors += acrpriors

Get max-likelihood starting point using scipy's minimize

In [ ]:
from scipy.optimize import minimize

In [ ]:
print("Before fit: logprob: {:.2f}, loglike: {:.2f}".format(acr_model_post.logprob(),acr_model_post.logprob()))

minresult = minimize(acr_model_post.neglogprob_array,acr_model_post.get_vary_params())

print("After fit: logprob: {:.2f}, loglike: {:.2f}".format(acr_model_post.logprob(),acr_model_post.logprob()))

Take a look at the fit

In [ ]:
from ResonantRV_Utils import plot_fit
plot_fit(acr_model_post.likelihood)

### 2.1 MCMC Fit

In [ ]:
acr_model_mcmc_results = radvel.mcmc(acr_model_post)

The cells below show the eccentricity posteriors of the ACR model. 
ACR tracks for the 1st and 99th percentile $m_2/m_1$ values are plotted for comparison.

In [ ]:
# Get planet eccentricities at posterior samples.
# Caution: iterrows is annoyingly slow...
e1,e2=[],[]
for i,x in  acr_model_mcmc_results[acr_model_post.likelihood.list_vary_params()].iterrows():
    acr_model_post.likelihood.set_vary_params(x)
    e1.append( acr_model_post.likelihood.model.get_synthparams()['e1'].value )
    e2.append( acr_model_post.likelihood.model.get_synthparams()['e2'].value )
acr_model_mcmc_results['e1'] = e1
acr_model_mcmc_results['e2'] = e2

In [ ]:
from corner import hist2d

from scipy.special import erf
fig,ax = plt.subplots(1)

hist2d(
    acr_model_mcmc_results['e1'].values,acr_model_mcmc_results['e2'].values,
    bins = 40,
    levels=[erf(i/np.sqrt(2)) for i in range(1,4)],
    plot_datapoints=False, plot_density=False,
    ax = ax    
)

for mratio in acr_model_mcmc_results.m2_by_m1.quantile((.01,0.99)):
    x,y = np.transpose([ acr_model_post.likelihood.model._acr_curves_fn(mratio,t) for t in np.linspace(0,1)])
    ax.plot(x,y,lw=3,color='red')
ax.set_xlim(0,0.4)
ax.set_ylim(0,0.4)

### 2.2 Nested sampling fit

Generate a (callable) prior transform object that defines a mapping from the unit hyper-cube to the parameters appearing in the likelihood model. The unit-hypercube is sampled uniformly by the nested sampling algorithm.

In [ ]:
acr_model_prior_transform = ACRModelPriorTransform(Observations,acr_model_like)

Get a dynesty NestedSample object

In [ ]:
sampler_acr_model = dynesty.NestedSampler(
    acr_model_like.logprob_array,
    acr_model_prior_transform,
    acr_model_prior_transform.Npars,
    sample='rwalk'
)

Run nested sampling

In [ ]:
_ = sampler_acr_model.run_nested()

Save the results of nested sampling run.

In [ ]:
acr_model_nested_results = sampler_acr_model.results

In [ ]:
acr_model_nested_results['logz'][-1]

# Model comparison

All that is left to do now is to compute the log of our Bayesian odds ratio for our two models:
$$
\log\left(\frac{{\cal Z}_{ACR}}{{\cal Z}_{full}}\right)
$$

In [ ]:
acr_model_nested_results['logz'][-1] - full_model_nested_results['logz'][-1]

In [ ]:
from ResonantRV_Utils import synthpars_to_sim

In [ ]:
sample =acr_model_mcmc_results.iloc[100]
acr_model_like.set_vary_params(
    sample[acr_model_like.list_vary_params()]
)
spars = acr_model_like.model.get_synthparams()
sim = synthpars_to_sim(spars,t0 = acr_model_like.model.time_base)
sim.move_to_com()

In [ ]:
vel = []
obs_times = acr_model_like.x
for time in obs_times:
    sim.integrate(time)
    vel.append(-1 * sim.particles[0].vz * 1731 * 1e3)


In [ ]:
plot_fit(acr_model_like)
ax = plt.gca()
ax.plot(obs_times,vel,'ro')